In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- OR summary ---
    if "OR" in d.columns:
        or_mean = d["OR"].mean()
        or_std = d["OR"].std(ddof=1)
        df_out.loc[feature, "OR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minOR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxOR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["OR"].mean()
    or_std  = d["OR"].std(ddof=1)
    df_out.loc[idx, "OR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minOR"] = d["2.5%"].min()
    df_out.loc[idx, "maxOR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
import time

t0 = time.perf_counter()


In [4]:
data = pd.read_csv('data/BBAG-cross.csv')

In [5]:
vars_ = ['Proficiency']


# Odds ratios

## Without co-vars

### Without iterations

In [6]:
results_merge_df_all = data.copy()

results_merge_df_all = results_merge_df_all.loc[:, ~results_merge_df_all.columns.duplicated()]

df_directions_odd = pd.DataFrame()

for i in vars_:
    
    y_ols = results_merge_df_all['GAP_bin']

    
    X_ols = results_merge_df_all[[i]]
    X_train, X_test, y_train, y_test = train_test_split(X_ols, y_ols, test_size=0.2, random_state=42)
    

    scaler = MinMaxScaler((0.05, 0.95))
    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    X_train_scaled['intercept'] = 1
    X_test_scaled['intercept'] = 1
    
    model = sm.Logit(y_train, X_train_scaled).fit(disp = 0)
    
    params = model.params
    conf = np.exp(model.conf_int())
    conf['OR'] = np.exp(params)
    conf['z'] =model.tvalues
    conf['P>|z|'] =model.pvalues
    conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']
    
    df = conf.loc[i:i]
    df['Feature'] = i
    df_directions_odd = pd.concat([df_directions_odd, df])

df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd.to_excel('Results/cross/cross_OR_-with-slovakia-proficiency.xlsx')
df_directions_odd

,2.5%,97.5%,OR,z,P>|z|,Feature
0,0.338345,0.400726,0.368217,-23.144257,1.660851e-118,Proficiency


### 1000-iteration

In [7]:


count = 0

results_dict = {}
for i in vars_:
    results_dict[i + "_y"] = pd.Series(dtype=float)
    results_dict[i + "_ypred"] = pd.Series(dtype=float)

results_merge_df_all = data.copy()
results_merge_df_all.dropna(subset=vars_ + ["GAP_bin"], inplace=True)
results_merge_df_all.reset_index(inplace=True, drop=True)

df_directions_odd = pd.DataFrame()

n_boosts = 10  #For performance reasons, the number of iterations was set to 10; however, 1,000 iterations were used for the results reported in the manuscript.”
test_size = 500 / results_merge_df_all.shape[0]

for boosts in range(n_boosts):
    for i in vars_:

        y_ols = results_merge_df_all["GAP_bin"]
        X_ols = results_merge_df_all[[i]]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols,
            test_size=test_size,
            stratify=results_merge_df_all["GAP_bin"],
            random_state=boosts
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled["intercept"] = 1
        X_test_scaled["intercept"] = 1


        model = sm.Logit(y_train, X_train_scaled).fit(disp=0)


        y_test_pred = model.predict(X_test_scaled)

        results_dict[i + "_y"] = pd.concat([results_dict[i + "_y"], y_test], axis=0)
        results_dict[i + "_ypred"] = pd.concat([results_dict[i + "_ypred"], y_test_pred], axis=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf["OR"] = np.exp(params)
        conf["z"] = model.tvalues
        conf["P>|z|"] = model.pvalues
        conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

        df = conf.loc[i:i].copy()
        df["Feature"] = i

        hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
        hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
        hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

        df["hl_statistic_10"] = hl10_stat
        df["hl_pvalue_10"] = hl10_p

        df["hl_statistic_50"] = hl50_stat
        df["hl_pvalue_50"] = hl50_p

        df["hl_statistic_100"] = hl100_stat
        df["hl_pvalue_100"] = hl100_p

        df_directions_odd = pd.concat([df_directions_odd, df], axis=0)

    df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd;


In [8]:
df_directions_odd_f = pd.DataFrame()

for f in vars_:
    df_directions_odd_f = summarize_feature(df_directions_odd, f, df_directions_odd_f, stat_cols=(10, 50, 100))

df_directions_odd_f.to_excel('Results/cross/cross_OR_-with-slovakia-proficiency-iter.xlsx')
df_directions_odd_f

,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR
Proficiency,0.14197,0.339029,0.990264,0.372011,0.370273,0.373749,0.341694,0.405001


## Odd ratios with macrosocials

### Without Iteration

In [9]:
vars_social_physical = ['gender_equal_l', 'Polution_conc_inv', 'Eq']

vars_sociopolitical = ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est']

In [10]:
results_merge_df_all = data.copy()

covar_list = (
    ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
     'number_leng_inst', 'number_stable_inst', 'distance', 'Migration']
    + vars_social_physical
    + vars_sociopolitical
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:

        y_ols = results_merge_df_all['GAP_bin']
        X_ols = results_merge_df_all[[i, covar]]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.Logit(y_train, X_train_scaled).fit(disp=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['OR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          # opcional pero útil
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [11]:
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    c_dfOR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-with-slovakia-proficiency_covar-' + covar + '.xlsx')

    print(covar)
    display(c_dfOR)

total_exposomes


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
0,0.320665,0.382216,0.35009,-23.43093,2.068625e-121,Proficiency,total_exposomes


sociopolitical


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
1,0.343449,0.406982,0.373868,-22.722061,2.711803e-114,Proficiency,sociopolitical


social_physical


...

,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
12,0.351466,0.418178,0.383373,-21.62447,1.057254e-103,Proficiency,inclu_suff_est


cred_elect_est


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
13,0.332555,0.394188,0.362062,-23.422477,2.522598e-121,Proficiency,cred_elect_est


local_dem_est


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
14,0.340033,0.402734,0.370058,-23.026052,2.556469e-117,Proficiency,local_dem_est


### 1000-Iteration

In [12]:

results_merge_df_all = data.copy()

covar_list = (
    ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
     'number_leng_inst', 'number_stable_inst', 'distance', 'Migration']
    + vars_social_physical
    + vars_sociopolitical
)



n_boosts = 10  #For performance reasons, the number of iterations was set to 10; however, 1,000 iterations were used for the results reported in the manuscript.”
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:

    for boosts in range(n_boosts):

        for i in vars_:

            y_ols = results_merge_df_all["GAP_bin"]
            X_ols = results_merge_df_all[[i, covar]]

            # Split igual al primero: test fijo ~500 y estratificado
            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.Logit(y_train, X_train_scaled).fit(disp=0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["OR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [13]:

features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100),
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [14]:
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    c_dfOR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-with-slovakia-proficiency_covar-' + covar + '-iter.xlsx')

    print(covar)
    display(c_dfOR)

total_exposomes


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
0,total_exposomes,Proficiency,0.055619,0.275505,0.969321,0.350022,0.348243,0.351801,0.320356,0.382564,10.0


sociopolitical


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
1,sociopolitical,Proficiency,0.050839,0.158483,0.966391,0.378298,0.376524,0.380072,0.347333,0.41195,10.0


social_physical


...

,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
12,inclu_suff_est,Proficiency,0.09155,0.248475,0.968023,0.386421,0.384442,0.388401,0.354095,0.421823,10.0


cred_elect_est


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
13,cred_elect_est,Proficiency,0.114262,0.255271,0.990542,0.364441,0.362689,0.366193,0.334631,0.397055,10.0


local_dem_est


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
14,local_dem_est,Proficiency,0.153568,0.202003,0.976816,0.373094,0.37134,0.374848,0.342646,0.406176,10.0


In [15]:
dt = time.perf_counter() - t0
m, s = divmod(dt, 60)
print(f"Tiempo total: {int(m)} min {s:.1f} s")

Tiempo total: 1 min 5.3 s
